In [2]:
using PETLION
using CSV
using DataFrames
using Plots
using Logging
using Dates

In [3]:
@time p = petlion(
    LCO;
    aging =:SEI,
    N_p = 10, # di=scretizations in the cathode
    N_s = 10, # discretizations in the separator
    N_n = 10, # discretizations in the anode
    N_r_p = 10, # discretizations in the solid cathode particles
    N_r_n = 10 , # discretizations in the solid anode particles
    temperature = true, # temperature enabled or disabled
    jacobian = :AD, # :AD (automatic-differenation) for convenience or :symbolic for speed

)

 64.764688 seconds (176.49 M allocations: 10.702 GiB, 2.69% gc time, 11.80% compilation time)


PETLION model{jacobian_AD}:
  Cathode: LCO, rxn_BV, & OCV_LCO
  Anode:   LiC6, rxn_BV, & OCV_LiC6
  System:  D_s_eff, rxn_rate, D_eff_linear, K_eff, & thermodynamic_factor_linear
  --------
  Temperature:     true
  Solid diffusion: Fickian, finite_difference
  Aging:           SEI
  Voltage bounds:  [2.5 V, 4.3 V]
  SOC bounds:      [0.0, 1.0]
  Temperature max: 55.0 °C
  --------
  N.a:   10
  N.p:   10
  N.r_p: 10
  N.s:   10
  N.n:   10
  N.r_n: 10
  N.z:   10

In [4]:
println(Dates.format(now(), "HH:MM")  )
println(pwd())

11:09


"/Users/wuzhengrun/Desktop/jupyter_notebooks/Sampling"

In [14]:
temperature = 35
p.θ[:T₀] = temperature+273.15 # 初始温度
p.θ[:T_amb] = temperature+273.15 # 环境温度
print(p)

PETLION model{jacobian_AD}:
  Cathode: LCO, rxn_BV, & OCV_LCO
  Anode:   LiC6, rxn_BV, & OCV_LiC6
  System:  D_s_eff, rxn_rate, D_eff_linear, K_eff, & thermodynamic_factor_linear
  --------
  Temperature:     true
  Solid diffusion: Fickian, finite_difference
  Aging:           SEI
  Voltage bounds:  [2.5 V, 4.3 V]
  SOC bounds:      [0.0, 1.0]
  Temperature max: 55.0 °C
  --------
  N.a:   10
  N.p:   10
  N.r_p: 10
  N.s:   10
  N.n:   10
  N.r_n: 10
  N.z:   10

In [22]:
m=0
n=9
j=0
empty!(p);PETLION.options[:FACTORIZATION_METHOD] = :LU
tempP = 350-0
# cp充电
V_max1 = 4.1
set_SOC_start = 0/10
set_SOC_end = 9/10

       
sol = simulate(p,  P=tempP, SOC=set_SOC_start,SOC_max = set_SOC_end, V_max=V_max1, outputs=:all)
simulate!(sol, p, V=:hold ,I_min=1/20,SOC_max = set_SOC_end)
start_0=1
set_SOH = sol.SOH[start_0] #  初始SOH
delta_SOH = sol.SOH[end] - sol.SOH[start_0]
SOC_end = sol.SOC[end]
# TODO：打印充放电曲线

for i in 0:3
    # cp放电

    simulate!(sol, p,P = -350, V_min=2.5, SOC_min = set_SOC_start) 
#     println("i放电第一流程P: ",i,"\tSOC_end:",sol.SOC[end])
    println(sol)

    simulate!(sol, p, V=:hold, SOC_min = set_SOC_start,I_max = -1/20)
    println("i放电第二流程V: ",i,"\tSOC_end:",sol.SOC[end])
    println(sol)
#     simulate!(sol, p, I=-1, SOC_min = set_SOC_start)
#     println("i放电第三流程I: ",i,"\tSOC_end:",sol.SOC[end])
#     println(sol)
    start_0=length(sol.t)+1
    t1=sol.t[end]
    # cp充电 SOC=set_SOC_start SOC不应该限制start，而是由上一步的放电得出
    simulate!(sol, p, P=tempP, V_max=V_max1, SOC_max = set_SOC_end, outputs=:all)
#     println("i充电第一流程P: ",i,"\tSOC_end:",sol.SOC[end])
    println(sol)
    simulate!(sol, p, V=:hold,SOC_max = set_SOC_end, I_min=1/20) 
    println("i充电第二流程V: ",i,"\tSOC_end:",sol.SOC[end])
    println(sol)
    start_0=length(sol.t)+1  
    if sol.SOH[end]<0.8
        println("Cycle Finish!\ttempP: ",tempP, "\ttemp_cycle: ", i,"\tm n",m," ",n)
        break
    end

end

PETLION simulation
  --------
  Runs:    P → V → P
  Time:    1577.81 s
  Current: -3.3388C
  Voltage: 3.5863 V
  Power:   -349.9964 W
  SOC:     0.585
  SOH:     0.9978
  Temp.:   55.0 °C
  Exit:    Above maximum permitted temperature
i放电第二流程V: 0	SOC_end:0.5850091940391016
PETLION simulation
  --------
  Runs:    P → V → P → V
  Time:    1577.85 s
  Current: -3.3407C
  Voltage: 3.5863 V
  Power:   -350.202 W
  SOC:     0.585
  SOH:     0.9978
  Temp.:   55.0014 °C
  Exit:    Above maximum permitted temperature
PETLION simulation
  --------
  Runs:    P → V → P → V → P
  Time:    1577.85 s
  Current: 3.1061C
  Voltage: 3.855 V
  Power:   350.0 W
  SOC:     0.585
  SOH:     0.9978
  Temp.:   55.0014 °C
  Exit:    Above maximum permitted temperature
i充电第二流程V: 0	SOC_end:0.5850091940391018
PETLION simulation
  --------
  Runs:    P → V → P → V → P → V
  Time:    1577.85 s
  Current: 3.1061C
  Voltage: 3.855 V
  Power:   350.0 W
  SOC:     0.585
  SOH:     0.9978
  Temp.:   55.0014 °C
  Exi

In [23]:
sol.t

194-element Vector{Float64}:
    0.0
    0.0033789350648329585
    0.006757870129665917
    0.013515740259331834
    0.027031480518663668
    0.054062961037327335
    0.07838026796915817
    0.10269757490098902
    0.12701488183281986
    0.1513321887646507
    0.1999668026283124
    0.2486014164919741
    0.2972360303556358
    ⋮
 1812.6771525719207
 1812.6811525719208
 1812.681152571921
 1812.6811525719213
 1812.6811525719215
 1812.6811525719218
 1812.681152571922
 1812.6811525719222
 1812.6811525719224
 1812.6811525719227
 1812.681152571923
 1812.6811525719231

In [68]:
plot(sol.t,sol.SOC,linewidth=2,title="My Plot")

In [69]:
sol.SOH[end]

0.7999671750904008

In [31]:
@time p2 = petlion(
    LCO;
    aging =:SEI,
    N_p = 10, # di=scretizations in the cathode
    N_s = 10, # discretizations in the separator
    N_n = 10, # discretizations in the anode
    N_r_p = 10, # discretizations in the solid cathode particles
    N_r_n = 10 , # discretizations in the solid anode particles
    temperature = false, # temperature enabled or disabled
    jacobian = :AD, # :AD (automatic-differenation) for convenience or :symbolic for speed

)

 10.955364 seconds (29.37 M allocations: 1.621 GiB, 2.12% gc time, 68.43% compilation time)


PETLION model{jacobian_AD}:
  Cathode: LCO, rxn_BV, & OCV_LCO
  Anode:   LiC6, rxn_BV, & OCV_LiC6
  System:  D_s_eff, rxn_rate, D_eff_linear, K_eff, & thermodynamic_factor_linear
  --------
  Temperature:     false
  Solid diffusion: Fickian, finite_difference
  Aging:           SEI
  Voltage bounds:  [2.5 V, 4.3 V]
  SOC bounds:      [0.0, 1.0]
  --------
  N.p:   10
  N.r_p: 10
  N.s:   10
  N.n:   10
  N.r_n: 10

In [36]:
m=0
n=9
j=0
empty!(p2);PETLION.options[:FACTORIZATION_METHOD] = :LU
tempP = 350-5*0
# cp充电
V_max1 = 4.1
set_SOC_start = 0/10
set_SOC_end = 9/10

       
sol = simulate(p2,  P=tempP, SOC=set_SOC_start,SOC_max = set_SOC_end, V_max=V_max1, outputs=:all)
simulate!(sol, p2, V=:hold ,I_min=1/20,SOC_max = set_SOC_end)
start_0=1
set_SOH = sol.SOH[start_0] #  初始SOH
delta_SOH = sol.SOH[end] - sol.SOH[start_0]
SOC_end = sol.SOC[end]
# TODO：打印充放电曲线
temp_cycle = 0
println("SOC_start:\n",sol.SOC[1])


i=0

# cp放电

simulate!(sol, p2, P=-tempP, V_min=2.5, SOC_min = set_SOC_start) 
println("i放电: ",i,"\tSOC_end:\n",sol.SOC[end])
simulate!(sol, p2, V=:hold, SOC_min = set_SOC_start)

println("i放电第二流程: ",i,"\tSOC_end:\n",sol.SOC[end])

start_0=length(sol.t)+1
t1=sol.t[end]
# cp充电 SOC=set_SOC_start SOC不应该限制start，而是由上一步的放电得出
simulate!(sol, p2, P=tempP, V_max=V_max1, SOC_max = set_SOC_end, outputs=:all)
println("i: ",i,"\tSOC_end:\n",sol.SOC[end])
simulate!(sol, p2, V=:hold,SOC_max = set_SOC_end, I_min=1/20)    
set_SOH = sol.SOH[start_0] #  初始SOH
delta_SOH = sol.SOH[end] - sol.SOH[start_0]
start_0=length(sol.t)+1  

println("i: ",i,"\tSOC_end:\n",sol.SOC[end])
if sol.SOH[end]<0.8
    println("Cycle Finish!\ttempP: ",tempP, "\ttemp_cycle: ", i,"\tm n",m," ",n)
end






SOC_start:
0.0
i放电: 0	SOC_end:
0.40348240090096243
i放电第二流程: 0	SOC_end:
-6.269747340605732e-8
i: 0	SOC_end:
0.5830681227959013
i: 0	SOC_end:
0.9001605422920766
